Цель данной работы - при помощи комментария, отзыва или любого другого текста, написанного пользователем, определять социотип по типологии Майерс-Бриггс. Это поможет лучше понимать пользователей и возможно поможет как дополнительный признак настроить рекомендательную систему.

Источник  https://habr.com/ru/company/surfingbird/blog/230103/

kaggle https://www.kaggle.com/datasnaek/mbti-type?select=mbti_1.csv

In [70]:
import pandas as pd

import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import itertools


comments = pd.read_csv("mbti_1.csv")
print(comments.shape)
comments['posts'][0]

(8675, 2)


"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

Как мы видим в постах много ссылок, лишних знаков, данные не очень чистые

In [71]:
comments.head(3)

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...


In [72]:
# !pip install razdel
# !pip install pymorphy2

In [73]:
#from gensim.test.utils import common_texts
#from gensim.corpora.dictionary import Dictionary

#предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [74]:
stopword_eng = stopwords.words('english')
print(len(stopword_eng))

morph = pymorphy2.MorphAnalyzer()

179


Дополнительно скачала стоп слова из свободных источников

In [75]:
with open('stop_words_english.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_eng += additional_stopwords
len(stopword_eng)

179

Также убрал сами названия типов из текста

In [76]:
list_of_types = ['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP']
for type in list_of_types:
    stopword_eng.append(type.lower())

In [77]:
len(stopword_eng)

195

In [78]:
f_1 = open("final_stop_words.txt", 'a', encoding="utf-8")
f_1.write(stopword_eng[0])
for word in stopword_eng[1:]:
    f_1.write('\n')
    f_1.write(word)
f_1.close()

In [79]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))
    text = re.sub("\|", ' ', str(text))
    text = re.sub("https?:\/\/\S+", '', str(text))
    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_eng] # [6]
    
    return words_lem_without_stopwords

Протестируем на 1 посте

In [80]:
text = clean_text(comments['posts'][0])
text2 = lemmatization(text)
print(text2)

['moments', 'sportscenter', 'top', 'ten', 'plays', 'pranks', 'lifechanging', 'experience', 'life', 'repeat', 'today', 'may', 'perc', 'experience', 'immerse', 'last', 'thing', 'friend', 'posted', 'facebook', 'committing', 'suicide', 'next', 'day', 'rest', 'peace', 'hello', 'sorry', 'hear', 'distress', 'natural', 'relationship', 'perfection', 'time', 'every', 'moment', 'existence', 'try', 'figure', 'hard', 'times', 'times', 'growth', 'welcome', 'stuff', 'game', 'set', 'match', 'prozac', 'wellbrutin', 'least', 'thirty', 'minutes', 'moving', 'legs', 'mean', 'moving', 'sitting', 'desk', 'chair', 'weed', 'moderation', 'maybe', 'try', 'edibles', 'healthier', 'alternative', 'basically', 'come', 'three', 'items', 'determined', 'type', 'whichever', 'types', 'want', 'would', 'likely', 'use', 'given', 'types', 'cognitive', 'functions', 'whatnot', 'left', 'things', 'moderation', 'sims', 'indeed', 'video', 'game', 'good', 'one', 'note', 'good', 'one', 'somewhat', 'subjective', 'completely', 'promoti

In [31]:
%%time
#Запускаем очистку текста
comments['posts'] = comments['posts'].apply(lambda x: clean_text(x), 1)

Wall time: 22.1 s


In [32]:
%%time
#Запускаем лемматизацию текста
comments['posts'] = comments['posts'].apply(lambda x: lemmatization(x), 1)

Wall time: 2min 22s


Посмотрим результат

In [33]:
comments['posts']

0       [moments, sportscenter, top, ten, plays, prank...
1       [finding, lack, posts, alarming, sex, boring, ...
2       [good, one, course, say, know, blessing, curse...
3       [dear, enjoyed, conversation, day, esoteric, g...
4       [fired, another, silly, misconception, approac...
                              ...                        
8670    [ixfp, always, think, cats, fi, doms, reason, ...
8671    [soif, thread, already, exists, someplace, els...
8672    [many, questions, things, would, take, purple,...
8673    [conflicted, right, comes, wanting, children, ...
8674    [long, since, personalitycafe, although, seem,...
Name: posts, Length: 8675, dtype: object

Создадим текст на основе очищенных слов (делаем join)

In [34]:
comments['posts'] = comments['posts'].apply(lambda x: ' '.join(x), 1)

In [35]:
comments['posts']

0       moments sportscenter top ten plays pranks life...
1       finding lack posts alarming sex boring positio...
2       good one course say know blessing curse absolu...
3       dear enjoyed conversation day esoteric gabbing...
4       fired another silly misconception approaching ...
                              ...                        
8670    ixfp always think cats fi doms reason especial...
8671    soif thread already exists someplace else heck...
8672    many questions things would take purple pill p...
8673    conflicted right comes wanting children honest...
8674    long since personalitycafe although seem chang...
Name: posts, Length: 8675, dtype: object

In [36]:
comments['type'].value_counts() # данные, к сожалению, не сбалансированы

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

In [37]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(comments, comments['type'], random_state=0)

#### Многоклассовая классификация с LogisticRegression

In [38]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

pipeline = Pipeline([('title_selector', FeatureSelector(column='posts')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('clf', LogisticRegression())])

In [39]:
#обучим наш пайплайн
pipeline.fit(X_train, y_train)

C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('title_selector', FeatureSelector(column='posts')),
                ('title_tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression())])

In [40]:
#наши прогнозы для тестовой выборки
preds = pipeline.predict(X_test)
preds[:10]

array(['INFP', 'INFP', 'ENFP', 'INTP', 'ENFP', 'INFP', 'INFP', 'INFJ',
       'INTJ', 'INFJ'], dtype=object)

In [41]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score

In [42]:
models_results = {
    'approach': [],
    'f1_score_micro': [],
    'f1_score_macro': [],
    'f1_score_weighted': [],
    'precision_score_micro': [],
    'precision_score_macro': [],
    'precision_score_weighted': []
}

In [43]:
models_results['approach'].append('Multi_LogReg')
models_results['f1_score_micro'].append(f1_score(y_test, preds, average='micro'))
models_results['f1_score_macro'].append(f1_score(y_test, preds, average='macro'))
models_results['f1_score_weighted'].append(f1_score(y_test, preds, average='weighted'))
models_results['precision_score_micro'].append(precision_score(y_test, preds, average='micro'))
models_results['precision_score_macro'].append(precision_score(y_test, preds, average='macro'))
models_results['precision_score_weighted'].append(precision_score(y_test, preds, average='weighted'))

print(models_results)

{'approach': ['Multi_LogReg'], 'f1_score_micro': [0.4693407100046104], 'f1_score_macro': [0.18337404414588507], 'f1_score_weighted': [0.411600059287563], 'precision_score_micro': [0.4693407100046104], 'precision_score_macro': [0.3241724895159938], 'precision_score_weighted': [0.46860787721084246]}


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Многоклассовая классификация с с GradientBoostingClassifier

In [44]:
from sklearn.ensemble import GradientBoostingClassifier

In [45]:
#соберем наш простой pipeline, но уже с другим классификатором
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

pipeline = Pipeline([('title_selector', FeatureSelector(column='posts')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('clf', GradientBoostingClassifier())])

In [46]:
#обучим наш пайплайн
pipeline.fit(X_train, y_train)

Pipeline(steps=[('title_selector', FeatureSelector(column='posts')),
                ('title_tfidf', TfidfVectorizer()),
                ('clf', GradientBoostingClassifier())])

In [47]:
#наши прогнозы для тестовой выборки
preds = pipeline.predict(X_test)
preds[:10]

array(['INTP', 'INFJ', 'ENFP', 'INTP', 'ENFP', 'INFP', 'INFP', 'ISFJ',
       'INTJ', 'INFJ'], dtype=object)

In [48]:
models_results['approach'].append('Multi_Grad_Boost')
models_results['f1_score_micro'].append(f1_score(y_test, preds, average='micro'))
models_results['f1_score_macro'].append(f1_score(y_test, preds, average='macro'))
models_results['f1_score_weighted'].append(f1_score(y_test, preds, average='weighted'))
models_results['precision_score_micro'].append(precision_score(y_test, preds, average='micro'))
models_results['precision_score_macro'].append(precision_score(y_test, preds, average='macro'))
models_results['precision_score_weighted'].append(precision_score(y_test, preds, average='weighted'))

print(models_results)

{'approach': ['Multi_LogReg', 'Multi_Grad_Boost'], 'f1_score_micro': [0.4693407100046104, 0.4698017519594283], 'f1_score_macro': [0.18337404414588507, 0.3160403370718312], 'f1_score_weighted': [0.411600059287563, 0.45972749476686353], 'precision_score_micro': [0.4693407100046104, 0.4698017519594283], 'precision_score_macro': [0.3241724895159938, 0.3509764738421239], 'precision_score_weighted': [0.46860787721084246, 0.4692340858698429]}


У GradientBoostingClassifier получились лучшие показатели

#### Многоклассовая классификация с RandomForestClassifier

In [49]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
#теперь соберем pipeline с RandomForestClassifier
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

pipeline = Pipeline([('title_selector', FeatureSelector(column='posts')), 
                     ('title_tfidf', TfidfVectorizer()), 
                     ('clf', RandomForestClassifier(n_estimators=1000))])

In [51]:
#обучим наш пайплайн и запишем прогнозы
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

In [52]:
models_results['approach'].append('Multi_RandFor')
models_results['f1_score_micro'].append(f1_score(y_test, preds, average='micro'))
models_results['f1_score_macro'].append(f1_score(y_test, preds, average='macro'))
models_results['f1_score_weighted'].append(f1_score(y_test, preds, average='weighted'))
models_results['precision_score_micro'].append(precision_score(y_test, preds, average='micro'))
models_results['precision_score_macro'].append(precision_score(y_test, preds, average='macro'))
models_results['precision_score_weighted'].append(precision_score(y_test, preds, average='weighted'))

print(models_results)

{'approach': ['Multi_LogReg', 'Multi_Grad_Boost', 'Multi_RandFor'], 'f1_score_micro': [0.4693407100046104, 0.4698017519594283, 0.3148916551406178], 'f1_score_macro': [0.18337404414588507, 0.3160403370718312, 0.084876394034293], 'f1_score_weighted': [0.411600059287563, 0.45972749476686353, 0.2316654944277123], 'precision_score_micro': [0.4693407100046104, 0.4698017519594283, 0.3148916551406178], 'precision_score_macro': [0.3241724895159938, 0.3509764738421239, 0.14820878482382907], 'precision_score_weighted': [0.46860787721084246, 0.4692340858698429, 0.33623489952208785]}


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Попробуем пойти другим путем. 16 бинарных подзадач с LogisticRegression

Вместо решения задачи многоклассовой классификации попробуем разделить на 16 бинарных задач и выберем тот тип по итогу, вероятность которого больше всех.

In [53]:
types = pd.get_dummies(comments['type'])
types.head()

,ENFJ,ENFP,ENTJ,ENTP,ESFJ,ESFP,ESTJ,ESTP,INFJ,INFP,INTJ,INTP,ISFJ,ISFP,ISTJ,ISTP
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [54]:
list_types = list(types.columns)
print(list_types)

['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP', 'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP']


In [55]:
comments_binary = pd.concat([comments['posts'], types], axis=1)
comments_binary.head()

,posts,ENFJ,ENFP,ENTJ,ENTP,ESFJ,ESFP,ESTJ,ESTP,INFJ,INFP,INTJ,INTP,ISFJ,ISFP,ISTJ,ISTP
0,moments sportscenter top ten plays pranks life...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,finding lack posts alarming sex boring positio...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,good one course say know blessing curse absolu...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,dear enjoyed conversation day esoteric gabbing...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,fired another silly misconception approaching ...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(comments_binary, comments_binary[list_types], random_state=0)

In [57]:
# в цикле обучим pipeline с LogisticRegression, добавляя новую колонку в  df_prob с вероятностью отнесения данного пользователя к определенному типу
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

df_prob = pd.DataFrame()

for i in list_types:
    
    pipeline = Pipeline([('title_selector', FeatureSelector(column='posts')), 
                         ('title_tfidf', TfidfVectorizer()), 
                         ('clf', LogisticRegression())])
    pipeline.fit(X_train, y_train[i])
    df_prob[i] = pipeline.predict_proba(X_test)[:,1]

y_test нам нужно вернуть в исходное состояние (до get_dummie) для сравнения с прогнозом. Для этого воспользуемся функцией decode.

In [58]:
def decode(row):
    for c in y_test.columns:
        if row[c] == 1:
            return c

In [59]:
y_test_multi = y_test.apply(decode, axis=1)
y_test_multi

4587    ISFP
2786    INFJ
2813    ENFP
3705    INTP
5957    ISFP
        ... 
7256    ISFP
2645    ENTP
4773    ISTP
7242    INFP
6523    INFJ
Length: 2169, dtype: object

In [60]:
df_prob.head()

,ENFJ,ENFP,ENTJ,ENTP,ESFJ,ESFP,ESTJ,ESTP,INFJ,INFP,INTJ,INTP,ISFJ,ISFP,ISTJ,ISTP
0,0.025947,0.091799,0.020118,0.036324,0.004075,0.006687,0.004117,0.008282,0.202597,0.288490,0.064519,0.090701,0.023790,0.028861,0.026699,0.028413
1,0.032001,0.121939,0.016113,0.040373,0.004232,0.004617,0.004070,0.007665,0.179996,0.295439,0.071861,0.071613,0.022333,0.029897,0.019405,0.021682
2,0.027614,0.226232,0.024436,0.031980,0.003790,0.005233,0.003825,0.011243,0.205206,0.203538,0.103852,0.022771,0.019201,0.029923,0.028727,0.038998
3,0.016235,0.029749,0.023017,0.062493,0.004084,0.004923,0.004498,0.008875,0.094269,0.078400,0.248879,0.374607,0.015797,0.017339,0.031989,0.045402
4,0.026481,0.180497,0.022176,0.072250,0.004276,0.006135,0.004372,0.011623,0.178760,0.184343,0.052230,0.071409,0.024485,0.039256,0.018921,0.037586


Теперь нам нужно выделить название колонки с максимальной вероятностью

In [61]:
df_prob['predict'] = df_prob.columns[df_prob.values.argsort(1)[:, -1]]
df_prob['predict']

0       INFP
1       INFP
2       ENFP
3       INTP
4       INFP
        ... 
2164    INFJ
2165    INFP
2166    INFP
2167    INFP
2168    INFJ
Name: predict, Length: 2169, dtype: object

In [62]:
models_results['approach'].append('Binary_LogReg')
models_results['f1_score_micro'].append(f1_score(y_test_multi, df_prob['predict'], average='micro'))
models_results['f1_score_macro'].append(f1_score(y_test_multi, df_prob['predict'], average='macro'))
models_results['f1_score_weighted'].append(f1_score(y_test_multi, df_prob['predict'], average='weighted'))
models_results['precision_score_micro'].append(precision_score(y_test_multi, df_prob['predict'], average='micro'))
models_results['precision_score_macro'].append(precision_score(y_test_multi, df_prob['predict'], average='macro'))
models_results['precision_score_weighted'].append(precision_score(y_test_multi, df_prob['predict'], average='weighted'))

print(models_results)

{'approach': ['Multi_LogReg', 'Multi_Grad_Boost', 'Multi_RandFor', 'Binary_LogReg'], 'f1_score_micro': [0.4693407100046104, 0.4698017519594283, 0.3148916551406178, 0.4601198709082526], 'f1_score_macro': [0.18337404414588507, 0.3160403370718312, 0.084876394034293, 0.1741265405364288], 'f1_score_weighted': [0.411600059287563, 0.45972749476686353, 0.2316654944277123, 0.39906927978331574], 'precision_score_micro': [0.4693407100046104, 0.4698017519594283, 0.3148916551406178, 0.4601198709082526], 'precision_score_macro': [0.3241724895159938, 0.3509764738421239, 0.14820878482382907, 0.32634445087824904], 'precision_score_weighted': [0.46860787721084246, 0.4692340858698429, 0.33623489952208785, 0.46937806070686355]}


C:\Users\User\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#### Теперь попробуем с GradientBoostingClassifier

In [63]:
X_train, X_test, y_train, y_test = train_test_split(comments_binary, comments_binary[list_types], random_state=0)

In [64]:
#Теперь также в цикле обучаем GradientBoostingClassifier
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

df_prob = pd.DataFrame()

for i in list_types:
    
    pipeline = Pipeline([('title_selector', FeatureSelector(column='posts')), 
                         ('title_tfidf', TfidfVectorizer()), 
                         ('clf', GradientBoostingClassifier())])
    pipeline.fit(X_train, y_train[i])
    df_prob[i] = pipeline.predict_proba(X_test)[:,1]

In [65]:
y_test_multi = y_test.apply(decode, axis=1)
df_prob.head()

,ENFJ,ENFP,ENTJ,ENTP,ESFJ,ESFP,ESTJ,ESTP,INFJ,INFP,INTJ,INTP,ISFJ,ISFP,ISTJ,ISTP
0,0.003845,0.045749,0.007157,0.027675,2.366944e-07,0.000002,2.577319e-07,0.00022,0.091358,0.259910,0.059663,0.122658,0.002593,0.00956,0.005242,0.015550
1,0.003845,0.037099,0.007157,0.027675,2.366944e-07,0.000002,2.577319e-07,0.00022,0.425386,0.094398,0.046834,0.150761,0.002593,0.00956,0.005242,0.015550
2,0.003845,0.626033,0.007157,0.027675,2.366944e-07,0.000002,2.577319e-07,0.00022,0.115132,0.132187,0.392216,0.053542,0.002593,0.00956,0.005242,0.015550
3,0.003845,0.032514,0.007157,0.027675,2.366944e-07,0.000002,2.577319e-07,0.00022,0.140021,0.127268,0.231436,0.522072,0.002593,0.00956,0.005242,0.027873
4,0.003845,0.246577,0.007157,0.033502,2.366944e-07,0.000002,2.577319e-07,0.00022,0.081303,0.101115,0.045844,0.062103,0.002593,0.00956,0.005242,0.015550


In [66]:
df_prob['predict'] = df_prob.columns[df_prob.values.argsort(1)[:, -1]]
df_prob['predict']

0       INFP
1       INFJ
2       ENFP
3       INTP
4       ENFP
        ... 
2164    ESTJ
2165    ENTP
2166    INFP
2167    INFP
2168    INFJ
Name: predict, Length: 2169, dtype: object

In [67]:
models_results['approach'].append('Binary_GradBoost')
models_results['f1_score_micro'].append(f1_score(y_test_multi, df_prob['predict'], average='micro'))
models_results['f1_score_macro'].append(f1_score(y_test_multi, df_prob['predict'], average='macro'))
models_results['f1_score_weighted'].append(f1_score(y_test_multi, df_prob['predict'], average='weighted'))
models_results['precision_score_micro'].append(precision_score(y_test_multi, df_prob['predict'], average='micro'))
models_results['precision_score_macro'].append(precision_score(y_test_multi, df_prob['predict'], average='macro'))
models_results['precision_score_weighted'].append(precision_score(y_test_multi, df_prob['predict'], average='weighted'))

In [68]:
models_results

{'approach': ['Multi_LogReg',
  'Multi_Grad_Boost',
  'Multi_RandFor',
  'Binary_LogReg',
  'Binary_GradBoost'],
 'f1_score_micro': [0.4693407100046104,
  0.4698017519594283,
  0.3148916551406178,
  0.4601198709082526,
  0.4564315352697095],
 'f1_score_macro': [0.18337404414588507,
  0.3160403370718312,
  0.084876394034293,
  0.1741265405364288,
  0.2968473525678659],
 'f1_score_weighted': [0.411600059287563,
  0.45972749476686353,
  0.2316654944277123,
  0.39906927978331574,
  0.44772701567633233],
 'precision_score_micro': [0.4693407100046104,
  0.4698017519594283,
  0.3148916551406178,
  0.4601198709082526,
  0.45643153526970953],
 'precision_score_macro': [0.3241724895159938,
  0.3509764738421239,
  0.14820878482382907,
  0.32634445087824904,
  0.32866995552654843],
 'precision_score_weighted': [0.46860787721084246,
  0.4692340858698429,
  0.33623489952208785,
  0.46937806070686355,
  0.4589796526188738]}

In [69]:
pd.DataFrame(data=models_results)

,approach,f1_score_micro,f1_score_macro,f1_score_weighted,precision_score_micro,precision_score_macro,precision_score_weighted
0,Multi_LogReg,0.469341,0.183374,0.411600,0.469341,0.324172,0.468608
1,Multi_Grad_Boost,0.469802,0.316040,0.459727,0.469802,0.350976,0.469234
2,Multi_RandFor,0.314892,0.084876,0.231665,0.314892,0.148209,0.336235
3,Binary_LogReg,0.460120,0.174127,0.399069,0.460120,0.326344,0.469378
4,Binary_GradBoost,0.456432,0.296847,0.447727,0.456432,0.328670,0.458980


В итоге для модели выбрал Multi_Grad_Boost.